# How to insert, update, and delete multiple records

To perform insert, update, and delete operations with multiple records on a database, you can use the <code>Execute</code> or <code>ExecuteAsync</code> methods of Dapper to run SQL statements or stored procedures with parameters. These methods return the number of affected rows by the operation. You can also use the <code>Query</code> or <code>QueryAsync</code> methods of Dapper to get the values from the inserted, updated, or deleted rows, by using clauses such as OUTPUT in your SQL statements. For example:



In [7]:
// Connection string
#load "AppSettings.cs"

In [8]:
#r "nuget:Microsoft.Data.SqlClient"
#r "nuget:System.ComponentModel"
#r "nuget:Dapper"

Installed Packages Dapper, 2.1.24 Microsoft.Data.SqlClient, 5.1.2 System.ComponentModel, 4.3.0

In [9]:
using Microsoft.Data.SqlClient;
using System;
using System.Collections.Generic;
using System.Data;
using Dapper;

public class Product
{    
    public int ProductID { get; set; }
    public string Name { get; set; }
    public string ProductNumber { get; set; }
    public string Color { get; set; }
    public decimal StandardCost { get; set; }
    public decimal ListPrice { get; set; }
    public int SafetyStockLevel { get; set; }
    public int ReorderPoint { get; set; }
    public int DaysToManufacture { get; set; }
    public DateTime SellStartDate { get; set; }
}

To insert new records into the Person.Person table, you can use the following code:

In [10]:
var rand = new Random();
var productsToInsert = new List<Product>()
{
    new Product() { Name = "Bike Helmet-" + rand.Next(1000, 9999), ProductNumber = "BK-H01-" + rand.Next(1000, 9999), Color = "Red", StandardCost = 15.50m, ListPrice = 34.99m, 
                    SafetyStockLevel = 999, ReorderPoint = 777, DaysToManufacture = 1, SellStartDate = new DateTime(2000, 1, 2) },    
    new Product() { Name = "Gloves-" + rand.Next(1000, 9999), ProductNumber = "GL-H01-" + rand.Next(1000, 9999), Color = "Black", StandardCost = 5.00m, ListPrice = 9.99m, 
                    SafetyStockLevel = 999, ReorderPoint = 777, DaysToManufacture = 1, SellStartDate = new DateTime(2000, 1, 2) }
};

List<Product> newPoducts;
using (var connection = new SqlConnection(connectionString))
{
    connection.Open();
    string sqlQuery = "INSERT INTO Production.Product (Name, ProductNumber, Color, StandardCost, ListPrice, SafetyStockLevel, ReorderPoint, DaysToManufacture, SellStartDate) VALUES (@Name, @ProductNumber, @Color, @StandardCost, @ListPrice, @SafetyStockLevel, @ReorderPoint, @DaysToManufacture, @SellStartDate)";
    connection.Execute(sqlQuery, productsToInsert);

    // Query the Product table and display the results   
    newPoducts = connection.Query<Product>("SELECT top 2 * FROM Production.Product order by ProductID desc").ToList();

    foreach (var product in newPoducts)
    {
        Console.WriteLine($"{product.ProductID}, {product.Name}, {product.ProductNumber}, {product.Color}, {product.StandardCost}, {product.ListPrice}");
    }
}

1086, Gloves-7588, GL-H01-2388, Black, 5.0000, 9.9900
1085, Bike Helmet-3249, BK-H01-3371, Red, 15.5000, 34.9900


To update records into the above table:

In [11]:
var productsToUpdate = new List<Product>()
{
    new Product() { ProductID = newPoducts[0].ProductID, Name = "Adjustable Race-" + rand.Next(1000, 9999), ProductNumber = "AR-" + rand.Next(1000, 9999), Color = "Silver", StandardCost = 300.00m, ListPrice = 400.00m,
                    SafetyStockLevel = 555, ReorderPoint = 747, DaysToManufacture = 0, SellStartDate = DateTime.Now },   
    new Product() { ProductID = newPoducts[1].ProductID, Name = "BB Ball Bearing-" + rand.Next(1000, 9999), ProductNumber = "BE-" + rand.Next(1000, 9999), Color = "Silver", StandardCost = 0.75m, ListPrice = 1.49m,
                    SafetyStockLevel = 555, ReorderPoint = 747, DaysToManufacture = 0, SellStartDate = DateTime.Now  }
};

using (var connection = new SqlConnection(connectionString))
{
    connection.Open();
    string sqlQuery = "UPDATE Production.Product SET Name = @Name, ProductNumber = @ProductNumber, Color = @Color, StandardCost = @StandardCost, ListPrice = @ListPrice, SafetyStockLevel = @SafetyStockLevel, ReorderPoint = @ReorderPoint, DaysToManufacture = @DaysToManufacture, SellStartDate = @SellStartDate WHERE ProductID = @ProductID";
    connection.Execute(sqlQuery, productsToUpdate);

    // Query the Product table and display the results   
    newPoducts = connection.Query<Product>("SELECT top 2 * FROM Production.Product order by ProductID desc").ToList();

    foreach (var product in newPoducts)
    {
        Console.WriteLine($"{product.ProductID}, {product.Name}, {product.ProductNumber}, {product.Color}, {product.StandardCost}, {product.ListPrice}");
    }
}

1086, Adjustable Race-7782, AR-8998, Silver, 300.0000, 400.0000
1085, BB Ball Bearing-6630, BE-5702, Silver, 0.7500, 1.4900


To delete above records from the above table:

In [12]:
using (var connection = new SqlConnection(connectionString))
{
    connection.Open();
    string sqlQuery = "DELETE FROM Production.Product WHERE ProductID IN @Ids";
    connection.Execute(sqlQuery, new { Ids = newPoducts.Select(s => s.ProductID).ToArray() });

    // Query the Product table and display the results
    var products = connection.Query<Product>("SELECT TOP 2 * FROM Production.Product order by ProductID desc").ToList();
    foreach (var product in products)
    {
        Console.WriteLine($"{product.ProductID}, {product.Name}, {product.ProductNumber}, {product.Color}, {product.StandardCost}, {product.ListPrice}");
    }
}


1070, Gloves-6630, GL-H01-4672, Black, 5.0000, 9.9900
1069, Bike Helmet-5504, BK-H01-9840, Red, 15.5000, 34.9900


Using Dapper’s <code>Query</code> and <code>Execute</code> method with parameters is a simple and convenient way to insert, update, and delete one or a list of objects, as it handles the mapping of the object properties to the SQL parameters automatically. However, this approach executes multiple INSERT statements sequentially for multiple objects, which can be slow and inefficient for large data sets. This approach also requires writing the SQL query manually, which can be prone to errors or typos.

If you need to work with a small or moderate amount of data occasionally, and you want to get the list of ids easily, you may prefer this approach. However, if you need to work with a large amount of data frequently, and you want to optimize the performance and network usage, you may prefer the bulk operations with <code>BulkInsert</code>, <code>BulkUpdate</code>, and <code>BulkDelete</code> classes.
